In [1]:
from schnetpack import AtomsData, AtomsLoader
from ase.visualize import view
import torch 
import numpy as np
from matplotlib import pyplot as plt
import h5py
from qml.representations import *
import sys
import pandas as pd
import time
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch import nn
from torch import optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn.functional as F
from VAE import base_VAE

2021-11-08 16:09:21.493516: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-08 16:09:21.493745: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
plt.rcParams["figure.figsize"] = (20, 20)

In [3]:
df=pd.read_json('./dataframe41537.json') #or whatever dataset you want to analyze

In [5]:
from separate_models import prop_ls_NN, prop_mol_NN

#load the direct model you prefer:
#'./main_testing/direct_global_1/trained_N' with N=0-14 for the global tested configurations
#'./main_testing/direct_local_{}/trained' with N=0-4 for the local in the order "hDIP", "vdWR", "atPOL","hCHG"
prop_list=list(df.columns[df.columns != 'BoB'])[8:53] #decide which properties you want
latent_size=32
input_dim=528

#load direct
PATH='./main_testing/direct_global_1/trained_0'
model_direct = prop_mol_NN(mol_size=input_dim,prop_size=len(prop_list),extra_size=32-len(prop_list))

#load the regular joint model (made of vae and mlp) you prefer:
#VAE part:
#'./main_testing/joint_global_1/vae_trained/trained_N' with N=0-14 for the global tested configurations
#'./main_testing/joint_local_N/vae_trained/trained' with N=0-4 for the local in the order "hDIP", "vdWR", "atPOL","hCHG"
#MLP part
#'./main_testing/joint_global_1/joint_trained/trained_N' with N=0-14 for the global tested configurations
#'./main_testing/joint_local_N/joint_trained/trained' with N=0-4 for the local in the order "hDIP", "vdWR", "atPOL","hCHG"

#load vae regular 
PATH='./main_testing/joint_global_1/vae_trained/trained_0'
model_vae = base_VAE(input_dim=input_dim, latent_size=latent_size)
model_vae.load_state_dict(torch.load(PATH))
#load mlp for regular vae
PATH='./main_testing/joint_global_1/joint_trained/trained_0' 
model_mlp_vae = prop_ls_NN(latent_size=latent_size,prop_size=len(prop_list))
model_mlp_vae.load_state_dict(torch.load(PATH))

#load the 'improved' joint model (made of vae and mlp) you prefer:
#VAE part:
#'./main_testing/joint_extra_global_1/vae_trained/trained_N' with N=0-14 for the global tested configurations
#'./main_testing/joint_local_extra_{}/vae_trained/trained' with N=0-4 for the local in the order "hDIP", "vdWR", "atPOL","hCHG"
#MLP part
#'./main_testing/joint_extra_global_1/joint_trained/trained_N' with N=0-14 for the global tested configurations
#'./main_testing/joint_local_extra_{}/joint_trained/trained' with N=0-4 for the local in the order "hDIP", "vdWR", "atPOL","hCHG"

#load vae+ 
PATH='./main_testing/joint_extra_global_1/vae_trained/trained_0'
model_vaep = base_VAE(input_dim=input_dim, latent_size=latent_size)
model_vaep.load_state_dict(torch.load(PATH))
#load mlp for vae+
PATH='./main_testing/joint_global_1/joint_trained/trained_0' 
model_mlp_vaep = prop_ls_NN(latent_size=latent_size,prop_size=len(prop_list))
model_mlp_vaep.load_state_dict(torch.load(PATH))

RuntimeError: Error(s) in loading state_dict for prop_ls_NN:
	Missing key(s) in state_dict: "model.0.weight", "model.0.bias", "model.2.weight", "model.2.bias", "model.4.weight", "model.4.bias". 
	Unexpected key(s) in state_dict: "model_mu.0.weight", "model_mu.0.bias", "model_mu.2.weight", "model_mu.2.bias", "model_mu.4.weight", "model_mu.4.bias", "model_logvar.0.weight", "model_logvar.0.bias", "model_logvar.2.weight", "model_logvar.2.bias", "model_logvar.4.weight", "model_logvar.4.bias". 

In [32]:
#here define a bunch of functions for dataset operations

def perc_error(a,b):
    a=torch.Tensor(a).abs()
    b=torch.Tensor(b)
    numer=torch.sum((a-b).abs())
    denom=torch.sum(b.abs())
    return 100*torch.mean(numer/denom)

def to_latent_mlp(x,model):
    model.eval()
    with torch.no_grad():
        x=torch.reshape(x,(1,x.size()[0]))
        u_reco,logvar=model(x)
    return u_reco

def to_latent_vae(x,model):
    model.eval()
    with torch.no_grad():
        x=torch.reshape(x,(1,x.size()[0]))
        u_reco,logvar=model.encode(x)
    return u_reco

def to_reco_vae (u, model):
    model.eval()
    with torch.no_grad():
        u=torch.reshape(u,(1,u.size()[0]))
        x_reco,logvar=model.decode(u)
    return x_reco

def to_reco_direct (x, model):
    model.eval()
        with torch.no_grad():
            x=torch.reshape(x,(1,x.size()[0]))
            x_reco,logvar=model(x)
        return x_reco


#full analysis, returns dataset with different latent representations and various errors (if it works since I never tested it)

def analysis(df, direct_model, vae_regular, mlp_regular, vae_plus, mlp_plus, prop_list):
    
    #for the direct model
    df['reconstructed_direct']=df[prop_list].apply(lambda x: to_reco_direct(torch.Tensor(x),direct_model)[0].tolist(),axis=1)
    df['error_reco_direct']=df[['reconstructed_direct','BoB']].apply(lambda x: perc_error(*x),axis=1)
    df['error_reco_direct']=df['error_reco_direct'].apply(lambda x: x.item())

    #for the standard vae joint implementation (both latent from vae and mlp and errors in reconstruction from vae and from mlp+vae)
    df['latent_vae_regular']=df[prop_list].apply(lambda x: to_latent_vae(torch.Tensor(x),vae_regular)[0].tolist(),axis=1)
    df['reconstructed_vae_regular']=df['latent_vae_regular'].apply(lambda x: to_reco(torch.Tensor(x),vae_regular).tolist())
    df['error_reco_vae_regular']=df[['reconstructed_vae_regular','BoB']].apply(lambda x: perc_error(*x),axis=1)
    df['error_reco_vae_regular']=df['error_reco_vae_regular'].apply(lambda x: x.item())

    df['latent_mlp_regular']=df[prop_list].apply(lambda x: to_latent_vae(torch.Tensor(x),mlp_regular)[0].tolist(),axis=1)
    df['reconstructed_joint_regular']=df['latent_mlp_regular'].apply(lambda x: to_reco(torch.Tensor(x),vae_regular).tolist())
    df['error_reco_joint_regular']=df[['reconstructed_joint_regular','BoB']].apply(lambda x: perc_error(*x),axis=1)
    df['error_reco_joint_regular']=df['error_reco_joint_regular'].apply(lambda x: x.item())


    #for the 'improved' vae joint implementation (both latent from vae and mlp and errors in reconstruction from vae and from mlp+vae)
    df['latent_vae_plus']=df[prop_list].apply(lambda x: to_latent_vae(torch.Tensor(x),vae_plus)[0].tolist(),axis=1)
    df['reconstructed_vae_plus']=df['latent_vae_plus'].apply(lambda x: to_reco(torch.Tensor(x),vae_plus).tolist())
    df['error_reco_vae_plus']=df[['reconstructed_vae_plus','BoB']].apply(lambda x: perc_error(*x),axis=1)
    df['error_reco_vae_plus']=df['error_reco_vae_plus'].apply(lambda x: x.item())

    df['latent_mlp_plus']=df[prop_list].apply(lambda x: to_latent_vae(torch.Tensor(x),mlp_plus)[0].tolist(),axis=1)
    df['reconstructed_joint_plus']=df['latent_mlp_plus'].apply(lambda x: to_reco(torch.Tensor(x),vae_plus).tolist())
    df['error_reco_joint_plus']=df[['reconstructed_joint_plus','BoB']].apply(lambda x: perc_error(*x),axis=1)
    df['error_reco_joint_plus']=df['error_reco_joint_plus'].apply(lambda x: x.item())

    return df


    




In [33]:
df_analyzed=analysis(df=df,direct_model=model_direct, vae_regular=model_vae, mlp_regular=model_mlp_vae, vae_plus=model_vaep, mlp_plus=model_mlp_vap, prop_list=prop_list)
df_analyzed

5.35655108698269

In [ ]:
#here you can do all the analysis you want, PCA in latent space, histograms, plots or whatever. When I have time I will set this stuff up